In [1]:
#pip install pandas

In [2]:
import pandas as pd
import time
from datetime import datetime
import math
import ccxt
import json

In [3]:
with open('binance.json') as f:
    binance_api = json.load(f)

In [4]:
exchange_id = 'binance'
exchange_class = getattr(ccxt, exchange_id)

exchange = exchange_class({
    "apiKey": "4MurciGOgsF8TI656k1ztYJi4UkygkJ0gHaKlbMG5fJzutQMJJBwPk6vbtBweMHA",
    "secret": "zuA7liNb8REGJO8c3ofRCGEkxMiAScInzcN5baPWuRJ21BfOIA3KfquPybILxAPY",
})

In [5]:
markets = exchange.fetchMarkets()
market_symbols = [market['symbol'] for market in markets if market['type']=='spot']

In [6]:
def create_chain(orders_type, start, middle, finish,  start_price, middle_price, finish_price, res):   
    return {
        'type': orders_type,
        'start': start['symbol'],
        'middle': middle['symbol'],
        'finish': finish['symbol'],
        'start_price': start_price,
        'middle_price': middle_price,
        'finish_price': finish_price,
        'Result' : res
    }
    

def get_chains(market_symbols, quotes, base_currency='USDT'):
    chains = []
    AMMOUNT=100.0

    for symbol in market_symbols:
        try:
            token1 = symbol.split('/')[0] 
            token2 = symbol.split('/')[1]
            #ищем промежуточное звено треугольника и проверяем результат в 2 стороны
            if (token1!=base_currency and token2!=base_currency):
                start = quotes[f'{token1}/{base_currency}']
                middle = quotes[f'{token1}/{token2}']
                finish = quotes[f'{token2}/{base_currency}']
                if  start['ask']!=0 and start['bid']!=0 and middle['ask']!=0 and middle['bid']!=0 and finish['ask']!=0 and finish['bid']!=0:
                    #USDT-ETH-BTC
                    chains.append(
                                create_chain(
                                    'BUY_SELL_SELL',
                                    start,         #ETH/USDT
                                    middle,        #ETH/BTC
                                    finish,        #BTC/USDT
                                    start['ask'],  #ETH/USDT ASK
                                    middle['bid'], #ETH/BTC BID
                                    finish['bid'], #BTC/USDT BID
                                    float(AMMOUNT/start['ask']*middle['bid']*finish['bid'] - AMMOUNT)
                                )
                    )
                    #USDT-BTC-ETH
                    chains.append(
                                create_chain(
                                    'BUY_BUY_SELL',
                                    finish,       #BTC/USDT
                                    middle,       #ETH/BTC
                                    start,        #ETH/USDT
                                    finish['ask'], #BTC/USDT ASK
                                    middle['ask'], ##ETH/BTC ASK
                                    start['bid'],  #ETH/USDT BID
                                    float(AMMOUNT/finish['ask']/middle['ask']*start['bid'] - AMMOUNT)
                                )
                   )

        except KeyError:
            pass
    return pd.DataFrame(chains)


In [7]:
def filter_profitable_chains(df, MIN_PROFIT, AMMOUNT):
    profit = df.loc[df['Result']>MIN_PROFIT]
    if not profit.empty:
        print(f"Time-{datetime.now().strftime('%H:%M:%S')}")
        print(profit)
        #разблокировать для отправки ордеров
        place_trade_orders(profit.iloc[0], AMMOUNT)

In [8]:
def place_trade_orders(chain, initial_amount):
    delta = 0.002 # увеличение/уменьшение цены лимитного ордера для возможного проскальзывания
    orders_type = chain['type']
    pair_1 = chain['start']
    pair_2 = chain['middle']
    pair_3 = chain['finish']
    price_1 = chain['start_price']
    price_2 = chain['middle_price']
    price_3 = chain['finish_price']
    
    order_1, order_2, order_3 = steps(orders_type, pair_1, pair_2, pair_3, price_1, price_2, price_3, initial_amount, delta)
        
    #ТОО Рассчитать убыток :(
    final_amount = 0.0
    
    time.sleep(10) #ждем 10 сек чтобы исключить возможный флипинг котировки
    return final_amount

In [9]:
def steps(orders_type, pair_1, pair_2, pair_3, price_1, price_2, price_3, initial_amount, delta):
    order_1, order_1_status = step_1(pair_1, price_1, initial_amount)
    order_2 = ""
    order_2_status = ""
    order_3 = ""
    quantity_3 = 0
    
    if order_1_status == 'closed':
        order_2, order_2_status, quantity_3 = step_2(orders_type, order_1['amount'], pair_2, pair_3, price_2, price_3, delta)
    elif order_1_status == 'part_filled':
        order_2, order_2_status, quantity_3 - step_2(orders_type, order_1['filled'], pair_2, pair_3, price_2, price_3, delta)
    #Вышли из цикла и приступаем к 3-му ордеру
    
    if order_2_status == 'closed' and quantity_3 != 0:
        order_3, order_3_status = step_3(pair_3, quantity_3, price_3, delta)
    elif order_2_status == 'reject':
        print("Нужно продать монеты в перой связке")
        
    return order_1, order_2, order_3

In [10]:
def step_1(pair_1, price_1, initial_amount):
    """
    TODO:
        Нужно проверить котировку... А оно нам надо??
        Если котировка изменилась и прибыли нет, Нас наебали... расходимся
        Нужно попробовать отменить ордер пока не поздно
        сделать проверку котировки
        Сделать проверку на частичное исполнение
        Ордер в первом запросе должен сразу сработать.Если не сработал я буду сразу отменять!
    """
    quantity_1 = initial_amount/price_1
    order = exchange.create_limit_buy_order(pair_1, quantity_1, price_1)
    status = order['status']
       
    if order['status'] == "open":
        if order['info']['status'] == "PARTIALLY_FILLED":
            order, status = part_filled_actions(order)
        else:
            order, status = cancel_order_actions(order)
    elif order['status'] == "closed":
        print ("Первый ордер исполнился")
        status = 'closed'
        
    print(order)
    return order, status

In [11]:
def step_2(orders_type, quantity_1, pair_2, pair_3, price_2, price_3, delta):
    # InsufficientFunds: binance Account has insufficient balance for requested action
    order = ""
    status = "new"
    quantity_3 = 0
    filled = 0
    left = 0
    
    #Изя, сколько будет 2*2?
    #А мы покупаем или продаем?
    #А сколько вам надо?
    if orders_type == 'BUY_BUY_SELL':
        quantity_2 = quantity_1/price_2
        order, status = send_limit_buy_order(pair_2, quantity_2, price_2)
        
        if status == "closed":
            filled = order['filled']
            left = 0
        elif status == "reject":
            print("Ордер 2 не исполнен и нужно попробовать вернуться к первой паре и сделать обратный обмен")
        elif status == "open":
            filled = 0
            left = order ['amount'] #binance округлит
            """
            Пока ордер не исполнится мы дальше не идем!
            Если мы не успели нужно попытаться догнать котировку и выйти с убытком!
            Второй вариант ждать когда котировка вернется и исполнит ордер.
            На практике это уже будет просто трейдинг.
            """
            while order['status'] != "closed":
                if order['status'] == "open":
                    order = exchange.fetch_order(order["id"], order["symbol"])
                    #Если ордер частично выполнен его лучше не отменять, а дождаться полного исполнения
                    #статус ордер будёт "ореп" но в инфо будет 'status': 'PARTIALLY FILLED'
                    #Нужно понимать что ордер может быть частично выполнен и эту ситуацию также нужно доработать
                    #ждем полного исполнения. Возможно добавление таймера или количества попыток
                    if order ['info']['status'] == "PARTIALLY_FILLED":
                        filled = filled + order['filled']
                        left = left - order ['filled']
                        for i in range (5):
                            time.sleep(0.5)
                            order = exchange.fetch_order(order["id"], order["symbol"])
                            if orderl['status'] == "closed":
                                break
                                
                    if order['status'] == "open":
                        order, _ = cancel_order_actions(order)
                        
                elif order['status'] == "canceled":
                    #При покупке нужно учитывать, что при увеличении цены нужно пропорционально уменьшать объем!
                    left = left/(1 + delta)
                    qty = left
                    price = order['price']*(1 + delta)
                    order, status_2 = send_limit_buy_order(pair_2, qty, price)
                    if status_2 == 'reject':
                        break
                        
                #проверка полного Выполнения ордера
                if order['status'] == "closed":
                    filled = filled + order['filled']
                    left = 0
                    status = 'closed'
                    break
                
        quantity_3 = filled
        print("quantity_3 = " +str(quantity_3))
    elif orders_type == 'BUY_SELL_SELL':
        quantity_2 = quantity_1/price_2
        order, status = send_limit_buy_order(pair_2, quantity_2, price_2)
        
        if status == "closed":
            filled = order['filled']
            left = 0
        elif status == "reject":
            print("Ордер 2 не исполнен и нужно попробовать вернуться к первой паре и сделать обратный обмен")
        elif status == "open":
            filled = 0
            left = order ['amount'] #binance округлит
            """
            Пока ордер не исполнится мы дальше не идем!
            Если мы не успели нужно попытаться догнать котировку и выйти с убытком!
            Второй вариант ждать когда котировка вернется и исполнит ордер.
            На практике это уже будет просто трейдинг.
            """
            while order['status'] != "closed":
                if order['status'] == "open":
                    order = exchange.fetch_order(order["id"], order["symbol"])
                    #Если ордер частично выполнен его лучше не отменять, а дождаться полного исполнения
                    #статус ордер будёт "ореп" но в инфо будет 'status': 'PARTIALLY FILLED'
                    #Нужно понимать что ордер может быть частично выполнен и эту ситуацию также нужно доработать
                    #ждем полного исполнения. Возможно добавление таймера или количества попыток
                    if order ['info']['status'] == "PARTIALLY_FILLED":
                        filled = filled + order['filled']
                        left = left - order ['filled']
                        for i in range (5):
                            time.sleep(0.5)
                            order = exchange.fetch_order(order["id"], order["symbol"])
                            if orderl['status'] == "closed":
                                break
                                
                    if order['status'] == "open":
                        order, _ = cancel_order_actions(order)
                        
                elif order['status'] == "canceled":
                    #При покупке нужно учитывать, что при увеличении цены нужно пропорционально уменьшать объем!
                    qty = left
                    price = order['price']/(1+delta)
                    order, status_2 = send_limit_sell_order(pair_2, qty, price)
                    if status_2 == 'reject':
                        break
                        
                if order['status'] == "closed":
                    filled = filled + order['filled']
                    left = 0
                    status = 'closed'
                    break
                
        quantity_3 = filled * order['average']
        print("quantity_3 = " +str(quantity_3))
        
    print(order)
    return order, status, quantity_3

In [12]:
def step_3(pair_3, quantity_3, price_3, delta):
    order, status = send_limit_sell_order(pair_3, quantity_3, price_3)
    filled = 0
    left = order['amount']
    
    if status == 'open':
        while order['status'] != "closed":
            if order['status'] == "open":
                time.sleep(0.5)
                order = exchange.fetch_order(order["id"], order["symbol"])
                if order['info']['status'] == "PARTIALLY_FILLED":
                    filled = filled + order['filled']
                    left = left - order['filled']
                    for i in range(5):
                        time.sleep(0.5)
                        order = exchange.fetch_order(order["id"], order["symbol"])
                        if order['status'] =="closed":
                            break
                            
                if order['status'] == "open":
                    order, _ = cancel_order_actions(order)
            elif order['status'] == "canceled":
                qty = left
                price = order['price']/(1 + delta)
                order, status = send_limit_sell_order(pair_3, qty, price)
                if status == 'reject':
                    break
                    
            if order['status'] == "closed":
                filled = filled + order['filled']
                left = 0
                status = 'closed'
                break
    elif status == 'reject':
            print ("Нужно проверить баланс!!!! "+order)
            
    print(order)
    return order, status
                

In [13]:
def send_limit_buy_order(pair, quantity, price):
    status = "new"
    order = ""
    try:
        order = exchange.create_limit_buy_order(pair, quantity, price)
        status = order ['status']
        print("Ордер отправлен "+order ['id'])
    except ccxt.InsufficientFunds:
        print("quantity=" + str(quantity) +", price = "+str(price))
        print ("Недостаточно средств проверьте выполнение ордеров")
        time.sleep(10)
        status = 'reject'
        pass 
    except ccxt.InvalidOrder:
        print("quantity=" + str(quantity) +", price = "+str(price))
        print("Не правильные параметры ордера")
        status = 'reject'
        pass
    return order, status

def send_limit_sell_order(pair, quantity, price):
    status = "new"
    order = ""
    try:
        order = exchange.create_limit_sell_order(pair, quantity, price)
        status = order['status']
        print("Ордер отправлен "+order['id'])
    except ccxt.InsufficientFunds:
        print("quantity=" + str(quantity) +", price = "+str(price))
        print("Недостаточно средств проверьте выполнение ордеров")
        time.sleep(10)
        status = 'reject'
        pass
    except ccxt.InvalidOrder:
        print("quantity=" + str(quantity) +", price = "+str(price))
        print("Не правильные параметры ордера"+order['id']+" - "+order ['symbol'])
        status = 'reject'
        pass
    return order, status

def part_filled_actions(order):
    status = order ['status']
    for i in range(5):
        time.sleep(0.5)
        order = exchange.fetch_order(order["id"], order["symbol"])
        if order ['status'] == "closed":
            status = "closed"
            break
            
    #Если после паузы ордер не исполнился полностью, то перейдем к следующему этапу на исполненую часть
    if order['status'] == "open":
        print("Первый ордер исполнился частично. Попробуем сделать связку на выполненный обьем "+order[ 'filled'])
        status = "part_filled"
        order, status_cancel_actions = cancel_order_actions(order)
        if status_cancel_actions == 'closed':
            status = "closed"
    return order, status

def cancel_order_actions(order):
    status = order ['status']
    try:
        order = exchange.cancel_order(order["id"], order['symbol'])
        print("Ордер отменен "+order['id']+" - "+order ['symbol'])
        if order['status'] == 'canceled':
            status = 'canceled'
    except ccxt.OrderNotFound:
        order = exchange.fetch_order(order["id"], order["symbol"])
        if order ['status'] == "closed":
            print ("Ордер таки исполнился"+order['id']+" - "+order['symbol'])
            status = 'closed'
    return order, status

In [14]:
MIN_PROFIT = 0.45
AMMOUNT = 100

while(1):    
    quotes = exchange.fetchBidsAsks()
    chains = get_chains(market_symbols, quotes, 'USDT')
    sorted_chains = chains.sort_values(by='Result', ascending=False)
    filter_profitable_chains(sorted_chains, MIN_PROFIT, AMMOUNT)
    time.sleep(1)

KeyboardInterrupt: 